In [ ]:
import numpy as np
import pandas as pd
import os

import gc
import matplotlib.pyplot as plt
import seaborn as sns

import scipy

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
import sklearn

In [ ]:
train = pd.read_pickle('../input/amexshycombinedfeatures/train.pkl')

In [ ]:
trainY = pd.read_pickle('../input/amexshycombinedfeatures/trainY.pkl')

In [ ]:
# Data Cleaning

In [ ]:
for i in train.columns:
    if train[i].dtypes == 'object':
        train[i] = train[i].apply(lambda x: (hash(str(x))+7)%10007)

In [ ]:
train['target'] = trainY

In [ ]:
train.info()

In [ ]:
# USE TWO SAMPLE K-S TEST TO TEST WHETHER LABEL 0/1 HAVE DIFFERENT DISTRIBUTION FOR A SPECIFIC FEATURE

In [ ]:
scores = []
# the null hypothesis is that the two distributions are identical
for i in train.columns:
    scores.append([i,scipy.stats.ks_2samp(train[train['target'] == 0][i],train[train['target'] == 1][i] , alternative='two-sided', mode='auto')[0]])
    print(scores[-1])

In [ ]:
# scores

In [ ]:
# import pickle

# with open(r"scores.pickle", "wb") as output:
#     pickle.dump(scores,output)

In [ ]:
import pickle
with open(r"../input/amexshycombinedfeatures/scores.pickle", "rb") as output:
    scores = pickle.load(output)

In [ ]:
scores.sort(key = lambda x: x[1])

In [ ]:
ksSelected = []
for i in range(0,800):
    ksSelected.append(scores[i][0])

In [ ]:
trainKsSelected = train[ksSelected]

In [ ]:
trainKsSelected.info()

In [ ]:
trainKsSelected.fillna(0,inplace=True)

In [ ]:
for i in trainKsSelected.columns:
    trainKsSelected[i] = trainKsSelected[i].replace([np.inf,-np.inf],0)

In [ ]:
trainKsSelected.to_pickle('trainKsSelected.pkl')

In [ ]:
trainKsSelected = pd.read_pickle('./trainKsSelected.pkl')

In [ ]:
# USE RANDOM FORESR AND RFECV TO SELECT 600 FEATURES

In [ ]:
# feature selection

import matplotlib.pyplot as plt
# from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier


# Create the RFE object and compute a cross-validated score.
# svc = SVC(kernel="linear")
# The "accuracy" scoring shows the proportion of correct classifications
rf = RandomForestClassifier(max_depth=6,max_features=30,max_samples= 0.05, random_state=0)

min_features_to_select = 600  # Minimum number of features to consider

rfecv = RFECV(
    estimator=rf,
    step=40,
    cv=StratifiedKFold(5),
    scoring="neg_root_mean_squared_error",
    min_features_to_select=min_features_to_select,
    verbose = 100
)

rfecv.fit(trainKsSelected, trainY)  

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (accuracy)")
plt.plot(
    range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select),
    rfecv.grid_scores_,
)   
plt.show()    

In [ ]:
selected1 = rfecv.get_support()
selected1 = rfecv.support_

In [ ]:
trainRFCVSelected = trainKsSelected.iloc[:,selected1]


In [ ]:
trainRFCVSelected.to_pickle('./trainRFCVSelected.pkl')

In [ ]:
trainRFCVSelected.head()

In [ ]:
test = pd.read_pickle('../input/amexshycombinedfeatures/test.pkl')
trainRFCVSelected = pd.read_pickle('../input/amexshycombinedfeatures/trainRFCVSelected.pkl')
columns = trainRFCVSelected.columns

In [ ]:
testRFCVSelected = test[columns]

In [ ]:
test = pd.read_pickle('../input/amexshycombinedfeatures/test.pkl')


In [ ]:
test.fillna(0,inplace=True)

In [ ]:
testRFCVSelected.fillna(0,inplace=True)

In [ ]:
for i in testRFCVSelected.columns:
    if testRFCVSelected[i].dtypes == 'object':
        testRFCVSelected[i] = testRFCVSelected[i].apply(lambda x: (hash(str(x))+7)%10007)

In [ ]:
for i in test.columns:
    if test[i].dtypes == 'object':
        test[i] = test[i].apply(lambda x: (hash(str(x)))%10007)

In [ ]:
for i in testRFCVSelected.columns:
    testRFCVSelected[i] = testRFCVSelected[i].replace([np.inf,-np.inf],0)

In [ ]:
for i in test.columns:
    test[i] = test[i].replace([np.inf,-np.inf],0)

In [ ]:
for i in testRFCVSelected.columns:
    if testRFCVSelected[i].dtypes == 'float64':
        testRFCVSelected[i] = testRFCVSelected[i].astype('float16')

In [ ]:
for i in test.columns:
    if test[i].dtypes == 'float64':
        test[i] = test[i].astype('float16')

In [ ]:
gc.collect()

In [ ]:
testRFCVSelected.info()

In [ ]:
testRFCVSelected.to_pickle('testRFCVSelected.pkl')

In [ ]:
test.info()

In [ ]:
test.to_pickle('testClean.pkl')